In [1]:
import numpy as np
import pandas as pd

# Integrantes
- Aguilar Martínez Erick Yair
- Martínez Muñoz Alan Magno
- Mendoza Hernández Carlos Emiliano

# Clasificador Bayesiano multinomial

**Probabilidad a posteriori**:
Es la probabilidad de que una observación pertenezca a una clase dada $y=k$ después de observar sus características, es decir, dado el vector de conteos
$x$ de cada característica. Usando el teorema de Bayes, la probabilidad a posteriori para una clase $y=k$ dado un vector de características $x=(x_1,x_2,...,x_n)$ se expresa como:

$$P(y=k|x)=\frac{P(x|y=k)\cdot P(y=k)}{P(x)}$$

Aquí:
- $P(y=k|x)$ es la probabilidad a posteriori de la clase $y=k$ dado el vector de características $x$.
- $P(x|y=k)$ es la probabilidad de ver el vector $x$ (probabilidad condicional) en la clase $y=k$. En el clasificador multinomial, esta probabilidad se calcula como la probabilidad de observar un conjunto de conteos específicos para las características en la clase $k$.
- $P(y=k)$ es la probabilidad a priori de la clase $k$.
- $P(x)$ es la probabilidad marginal de observar el vector $x$ en cualquier clase. Como $P(x)$ es constante para todas las clases, se puede omitir al comparar entre clases (no afecta la selección de la clase con mayor probabilidad).

Para simplificar el cálculo y evitar problemas numéricos, trabajamos con logaritmos. La expresión en logaritmos (omitimos $P(x)$ porque es constante) es:

$$\log P(y=k|x) \propto \log P(y=k) + \sum_{j=1}^{n} x_j \cdot \log P(x_j|y=k) $$

Donde:

- $\log P(y=k)$ es el logaritmo de la probabilidad a priori de la clase $k$
- $\sum_{j=1}^{n}x_j \cdot \log P(x_j|y=k)$ es la suma de las contribuciones de cada característica al logaritmo de la probabilidad condicional, ponderada por el conteo $x_j$ de cada característica en el vector $x$.

**Probabilidad a priori**: La probabilidad a priori en el contexto de un clasificador bayesiano multinomial es la probabilidad de que una observación pertenezca a una clase específica antes de ver las características de la observación. Esta probabilidad se basa en la frecuencia de las clases en el conjunto de datos de entrenamiento.

La expresión matemática de la probabilidad a priori para una clase $y=k$
$$P(y=k) = \frac{\text{Número de observaciones en la clase }k}{\text{Número total de observaciones}}$$

Dado que estamos trabajando con logaritmos para evitar problemas de precisión numérica, en la implementación usualmente calculamos el logaritmo de esta probabilidad, es decir:

$$\log P(y=k) = \log \left( \frac{\text{Número de observaciones en la clase }k}{\text{Número total de observaciones}} \right)$$

**Probabilidades condicionales de las características dada una clase**:
Las probabilidades condicionales de las características dada una clase en el clasificador bayesiano multinomial se refieren a la probabilidad de observar un conteo específico de una característica $x_j$ en una clase $y=k$. Estas probabilidades representan que tan probable es cada característica dentro de una clase en función de la frecuencia relativa de esa característica en los datos de entrenamiento.

La expresión matemática para la probabilidad condicional de cada característica $x_j$ dado que la clase es $y=k$ es:

$$P(x_j|y=k)=\frac{\text{Número de ocurrencias de } x_j \text{ en la clase }k}{\text{Número total de ocurrencias de todas las características de la clase }k}$$

Para evitar problemas de precisión numérica, calculamos el logaritmo de las probabilidades condicionales. La expresión en logaritmo para $P(x_j|y=k)$ es entonces:

$$ \log P(x_j|y=k)= \log \left( \frac{\text{Número de ocurrencias de } x_j \text{ en la clase }k}{\text{Número total de ocurrencias de todas las características de la clase }k} \right)$$

**Predicción**:

Se evalúa la expresión de la probabilidad a posteriori para cada clase, y se selecciona la clase con el mayor valor de $\log P(y=k∣x)$ como la predicción.

$$\hat{y}=\text{arg máx} _k \left\{ \log P(y=k) + \sum_{j=1}^{n} x_j \cdot \log P(x_j|y=k) \right\}$$

$$\mathbf{X} \Rightarrow \text{ matriz de contadores}$$ 
$$\mathbf{y} \Rightarrow \text{ vector de clases para cada observación en }\mathbf{X}$$

In [ ]:
class MultinomialNB:
    """
    Multinomial Naive Bayes classifier.
    """

    def __init__(self) -> None:
        """
        Initialize the model.
        """
        self.clases = None
        self.cont_carac = None
        self.prob_priori_log = None
        self.prob_cond_carac_log = None

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        """
        Fit the model to the data.

        Parameters:
        -----------
        X: np.array
            Count matrix of shape (n_samples, n_features)
        y: np.array
            Class labels of shape (n_samples,)
        Returns:
        --------
        np.array
            Predicted class labels of shape (n_samples,)
        """
        # Probabilidad a priori de las clases
        self.clases, counts = np.unique(y, return_counts=True)
        # log (conteo de cada clase / total de observaciones)
        self.prob_priori_log = np.log(counts / len(y))
        # Probabilidad condicional de las características
        self.cont_carac = np.zeros((len(self.clases), X.shape[1]))
        for i, c in enumerate(self.clases):
            X_c = X[y == c]
            self.cont_carac[i, :] = np.sum(X_c, axis=0) + 1  # Suma 1 para evitar ceros
        # log (conteo de las características en la clase / total de todos lso conteos en la clase)
        self.prob_cond_carac_log = np.log(self.cont_carac / self.cont_carac.sum(axis=1, keepdims=True))

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Predict the class labels.

        Parameters:
        -----------
        X: np.array
            Count matrix of shape (n_samples, n_features)
        Returns:
        --------
        np.array
            Predicted class labels of shape (n_samples,)
        """
        # Formulazo
        probas = X @ self.prob_cond_carac_log.T + self.prob_priori_log
        k_max = self.clases[np.argmax(probas, axis=1)]
        return k_max

**Ejemplos**:

In [3]:
model = MultinomialNB()
X = np.array([[2, 1, 0], [1, 3, 1], [0, 2, 4]])
y = np.array([0, 1, 1])
# Ajuste
model.fit(X, y)
# Predicción
X_new = np.array([[1, 2, 0]])
y_pred = model.predict(X_new)
print(y_pred)

[0]


Ejemplo SPAM (visto en clase con sklearn)

In [4]:
df = pd.read_csv('spam.csv', names=['label', 'sms_message'])
df.head()

,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
# Preprocessing
df.label = df.label.map({'ham':0, 'spam':1})
df.head()

,label,sms_message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
# Split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['sms_message'], df['label'], random_state=1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4179,), (1393,), (4179,), (1393,))

In [7]:
# CV
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
train_vec = cv.fit_transform(X_train)
test_vec = cv.transform(X_test)

In [8]:
# MultinomialNB
modelo = MultinomialNB()
modelo.fit(train_vec, y_train)

In [9]:
y_hat = modelo.predict(test_vec)

Metricas del modelo

In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(y_test, y_hat)
precision = precision_score(y_test, y_hat)
recall = recall_score(y_test, y_hat)
f1 = f1_score(y_test, y_hat)
accuracy, precision, recall, f1

(0.9856424982053122,
 0.9545454545454546,
 0.9333333333333333,
 0.9438202247191011)